In [ ]:
import cv2

# Load the pre-trained Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale (faster processing)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Display the output
    cv2.imshow('Face Detection', frame)
    
    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Load the pre-trained Caffe model (offline)
model_weights = r"C:\Users\Hydra\OneDrive\Desktop\Projects\Face_detection_attendance\res10_300x300_ssd_iter_140000.caffemodel"
model_config = r"C:\Users\Hydra\OneDrive\Desktop\Projects\Face_detection_attendance\deploy.prototext"
net = cv2.dnn.readNetFromCaffe(model_config, model_weights)

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Reduce resolution for speed
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    (h, w) = frame.shape[:2]
    
    # Preprocess frame for the DNN
    blob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)),  # Resize to 300x300 for the model
        1.0,                            # Scale factor
        (300, 300),                     # Input size
        (104.0, 177.0, 123.0)          # Mean subtraction (RGB)
    )
    
    # Run face detection
    net.setInput(blob)
    detections = net.forward()

    # Loop over detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        # Filter weak detections (confidence > 70%)
        if confidence > 0.7:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            
            # Ensure bounding box stays within frame
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w - 1, x2), min(h - 1, y2)
            
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Face: {confidence:.2f}", (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display output
    cv2.imshow("DNN Face Detection", frame)
    
    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import time

# Load Haar Cascade (bundled with OpenCV, works offline)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

# Initialize webcam with reduced resolution
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Lower width for faster processing
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Lower height

# Frame skipping to reduce load
frame_skip = 2  # Process every 3rd frame
frame_counter = 0

# FPS counter variables
prev_time = 0
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    if frame_counter % frame_skip != 0:
        continue  # Skip this frame

    # Convert to grayscale (faster processing)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces with optimized parameters
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,  # Slightly reduce scaling (faster)
        minNeighbors=6,    # Increase to reduce false positives
        minSize=(50, 50)   # Larger min size skips small detections
    )

    # Draw rectangles
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Calculate FPS
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display
    cv2.imshow("Haar Cascade Face Detection (Optimized)", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import os

# Initialize Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create datasets folder
os.makedirs("datasets", exist_ok=True)

# Input student name
student_name = input("Enter student name (e.g., 'john_doe'): ").strip().lower()
student_dir = f"datasets/{student_name}"
os.makedirs(student_dir, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

count = 0
while count < 5:  # Capture 5 images per student
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        face_roi = gray[y:y+h, x:x+w]
        cv2.imwrite(f"{student_dir}/{count}.jpg", face_roi)
        print(f"Saved {student_name}_{count}.jpg")
        count += 1
        cv2.waitKey(300)  # Short delay between captures

    cv2.putText(frame, f"Capturing: {student_name} ({count}/5)", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.imshow("Press 'q' to quit", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import os
import numpy as np

# Initialize face recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Prepare training data
faces = []
labels = []
label_ids = {}
current_id = 0

# Scan dataset folder
for root, dirs, files in os.walk("datasets"):
    for dir_name in dirs:
        student_name = dir_name
        if student_name not in label_ids:
            label_ids[student_name] = current_id
            current_id += 1

        for file in os.listdir(f"{root}/{dir_name}"):
            if file.endswith(".jpg"):
                img_path = os.path.join(root, dir_name, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                faces.append(img)
                labels.append(label_ids[student_name])

# Train and save model
recognizer.train(faces, np.array(labels))
os.makedirs("trainer", exist_ok=True)
recognizer.write("trainer/trainer.yml")

# Save label mappings
with open("trainer/labels.txt", "w") as f:
    for name, id in label_ids.items():
        f.write(f"{name}:{id}\n")

print(f"Trained model for {len(label_ids)} students: {list(label_ids.keys())}")

In [ ]:
import cv2
import os
import time

# Load trained model and labels
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer/trainer.yml")

# Load label mappings
label_ids = {}
with open("trainer/labels.txt", "r") as f:
    for line in f.readlines():
        name, id = line.strip().split(":")
        label_ids[int(id)] = name

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Track attendance
attendance_log = set()

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        id_, confidence = recognizer.predict(roi_gray)

        if confidence < 70:  # Confidence threshold
            student_name = label_ids[id_]
            cv2.putText(frame, student_name, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            attendance_log.add(student_name)
        else:
            cv2.putText(frame, "Unknown", (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display attendance
    cv2.putText(frame, f"Present: {', '.join(attendance_log)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Press 'q' to quit", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save to CSV
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
with open(f"attendance_{timestamp}.csv", "w") as f:
    f.write("Name,Time\n")
    for name in attendance_log:
        f.write(f"{name},{timestamp}\n")

cap.release()
cv2.destroyAllWindows()

In [ ]:
from flask import Flask, request, jsonify, send_file, Response  # dummy server
import sqlite3  #data base
import numpy as np #image to numpy array
import os
import io
from datetime import datetime, date
from flask_cors import CORS
from PIL import Image # deals with image such as  feature extraction
from deepface import DeepFace # face recognition model
from scipy.spatial.distance import cosine 

app = Flask(__name__)
CORS(app)

# --- Configuration for DeepFace ---
DEEPFACE_MODEL = 'VGG-Face' # You can try 'FaceNet', 'OpenFace', 'ArcFace', etc.
                           # 'VGG-Face' is a good general choice.
DEEPFACE_THRESHOLD = 0.45 # Cosine similarity threshold for VGG-Face. Needs tuning.
                          # Smaller value (closer to 0) means more strict match.
                          # Typical range is 0.0 to 1.0 for cosine distance.
                          # Common thresholds: VGG-Face (0.45-0.6), FaceNet (0.7-0.8)

# Database initialization with improved schema
def init_db():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()

    # Create tables if they don't exist
    c.execute('''CREATE TABLE IF NOT EXISTS students
                 (student_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  first_name TEXT NOT NULL,
                  last_name TEXT NOT NULL,
                  email TEXT,
                  course TEXT,
                  year INTEGER,
                  face_encoding BLOB,
                  registration_date TEXT,
                  image BLOB)''')  # Added image storage

    c.execute('''CREATE TABLE IF NOT EXISTS attendance
                 (attendance_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  student_id INTEGER,
                  date TEXT,
                  status TEXT, -- 'present', 'absent', 'late'
                  time TEXT,
                  image BLOB, 
                  FOREIGN KEY(student_id) REFERENCES students(student_id))''')

    conn.commit()
    conn.close()

init_db()

# Helper functions for face encodings (compatible with DeepFace embeddings)
def face_encoding_to_blob(encoding):
    """Converts a numpy array face encoding to a bytes object for DB storage.
    Ensures the encoding is float32 to prevent shape mismatches."""
    return np.array(encoding, dtype=np.float32).tobytes() # Explicitly cast to float32

def blob_to_face_encoding(blob):
    """Converts a bytes object from DB back to a numpy array face encoding.
    Ensures correct dtype for DeepFace embeddings (float32)."""
    return np.frombuffer(blob, dtype=np.float32)

def load_known_faces():
    """Load all registered face encodings from database."""
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT student_id, face_encoding FROM students WHERE face_encoding IS NOT NULL")
    rows = c.fetchall()
    conn.close()

    known_encodings = []
    known_ids = []
    for student_id, encoding_blob in rows:
        if encoding_blob:
            # Ensure correct dtype for DeepFace embeddings (often float32)
            encoding_array = np.frombuffer(encoding_blob, dtype=np.float32)
            known_encodings.append(encoding_array)
            known_ids.append(student_id)
    return known_encodings, known_ids

def compare_deepface_encodings(known_encodings, unknown_encoding, threshold=DEEPFACE_THRESHOLD):
    """
    Compares an unknown DeepFace encoding against a list of known DeepFace encodings.
    Returns a list of booleans indicating matches based on cosine distance.
    """
    matches = []
    for known_encoding in known_encodings:
        # Cosine distance: lower value indicates higher similarity.
        # If distance is less than the threshold, it's considered a match.
        distance = cosine(known_encoding, unknown_encoding)
        matches.append(distance < threshold)
    return matches

# API Endpoints
@app.route('/register_student', methods=['POST'])
def register_student():
    """Enhanced registration with better face detection using DeepFace."""
    try:
        data = request.form
        image_file = request.files.get('image')

        if not image_file:
            return jsonify({'error': 'No image file provided'}), 400

        # Read image file into memory and convert to numpy array for DeepFace
        img_bytes = image_file.read()
        pil_image = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        image_np = np.array(pil_image)

        # Detect faces using DeepFace's extract_faces (returns bounding boxes)
        # Using 'opencv' detector as it's generally robust and widely available.
        # enforce_detection=False allows the function to return empty list if no face is found
        detected_faces = DeepFace.extract_faces(
            img_path=image_np,
            detector_backend='opencv',
            enforce_detection=False
        )

        if not detected_faces: # No faces found
            return jsonify({'error': 'No face detected in the image'}), 400
        elif len(detected_faces) > 1:
            return jsonify({'error': 'Multiple faces detected. Please upload an image with a single face.'}), 400

        # Get the single detected face's bounding box and crop it
        face_area = detected_faces[0]['facial_area']
        x, y, w, h = face_area['x'], face_area['y'], face_area['w'], face_area['h']
        cropped_face_np = image_np[y:y+h, x:x+w]

        # Get face encoding for the cropped face using DeepFace.represent
        # This generates the numerical vector for the face.
        # enforce_detection=False because we already detected and cropped.
        embeddings = DeepFace.represent(
            img_path=cropped_face_np,
            model_name=DEEPFACE_MODEL,
            enforce_detection=False
        )
        if not embeddings: # Should not happen if detected_faces had one entry
            return jsonify({'error': 'Could not generate face embedding.'}), 500

        face_encoding = embeddings[0]['embedding']
        face_blob = face_encoding_to_blob(face_encoding) # Use the fixed function here

        # Check if this face already exists in the database
        known_face_encodings, _ = load_known_faces()
        if known_face_encodings:
            matches = compare_deepface_encodings(known_face_encodings, face_encoding, threshold=DEEPFACE_THRESHOLD)
            if any(matches):
                return jsonify({'error': 'This face is already registered in the system'}), 400

        # Save to database
        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()

        c.execute('''INSERT INTO students
                     (first_name, last_name, email, course, year, face_encoding, registration_date, image)
                     VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                     (data.get('first_name'), data.get('last_name'), data.get('email'),
                      data.get('course'), data.get('year'), face_blob, datetime.now().isoformat(), img_bytes))

        student_id = c.lastrowid
        conn.commit()
        conn.close()

        return jsonify({
            'success': True,
            'student_id': student_id,
            'message': 'Student registered successfully with face data'
        })

    except sqlite3.IntegrityError as e:
        print(f"Database error during registration: {e}")
        return jsonify({'error': 'Database error. Student may already exist.'}), 400
    except Exception as e:
        print(f"Error during student registration: {e}")
        return jsonify({'error': f'An internal error occurred during registration: {str(e)}'}), 500

@app.route('/mark_attendance', methods=['POST'])
def mark_attendance():
    """Enhanced attendance marking using DeepFace."""
    try:
        image_file = request.files.get('image')
        if not image_file:
            return jsonify({'error': 'No image file provided'}), 400

        # Read image file and convert to numpy array for DeepFace
        img_bytes = image_file.read()
        pil_image = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        unknown_image_np = np.array(pil_image)

        # Detect all faces in the unknown image
        detected_faces_info = DeepFace.extract_faces(
            img_path=unknown_image_np,
            detector_backend='opencv',
            enforce_detection=False
        )

        if not detected_faces_info: # No faces found
            return jsonify({'recognized': [], 'message': 'No faces detected in the image'}), 200

        unknown_face_encodings = []
        for face_info in detected_faces_info:
            x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']
            cropped_face_np = unknown_image_np[y:y+h, x:x+w]
            
            # Get encoding for each detected face
            embeddings = DeepFace.represent(
                img_path=cropped_face_np,
                model_name=DEEPFACE_MODEL,
                enforce_detection=False # Already detected, so disable internal detection
            )
            if embeddings: # Ensure an embedding was actually produced
                unknown_face_encodings.append(embeddings[0]['embedding'])


        if not unknown_face_encodings: # No recognizable faces found after attempting to get embeddings
             return jsonify({'recognized': [], 'message': 'No recognizable faces found in the image'}), 200

        # Get all registered students' DeepFace embeddings
        known_face_encodings, known_student_ids = load_known_faces()

        # Get additional student info from DB
        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()
        c.execute('SELECT student_id, first_name, last_name, course FROM students')
        students_data = {row[0]: (row[1], row[2], row[3]) for row in c.fetchall()}
        conn.close()

        recognized_students_info = []
        marked_attendance_count = 0
        current_date = date.today().isoformat()
        current_time = datetime.now().strftime("%H:%M:%S")

        # Compare unknown faces with known faces
        for unknown_encoding in unknown_face_encodings:
            student_id = None
            student_name = "Unknown"
            course = "Unknown"
            status = "Not Recognized" # Default status if no match

            if known_face_encodings:
                # Find the best match by calculating distances to all known embeddings
                distances = [cosine(known_encoding, unknown_encoding) for known_encoding in known_face_encodings]
                min_distance_index = np.argmin(distances)
                min_distance = distances[min_distance_index]

                if min_distance < DEEPFACE_THRESHOLD: # If the best match is within the threshold
                    student_id = known_student_ids[min_distance_index]
                    if student_id in students_data:
                        first_name, last_name, course = students_data[student_id]
                        student_name = f"{first_name} {last_name}"
                        
                        # Check if attendance already marked today for this student
                        conn = sqlite3.connect('attendance.db')
                        c = conn.cursor()
                        c.execute('SELECT attendance_id FROM attendance WHERE student_id = ? AND date = ?',
                                  (student_id, current_date))
                        existing_attendance = c.fetchone()
                        conn.close() # Always close the connection

                        if existing_attendance is None:
                            # Mark attendance
                            conn = sqlite3.connect('attendance.db')
                            c = conn.cursor()
                            c.execute('''INSERT INTO attendance
                                         (student_id, date, status, time, image)
                                         VALUES (?, ?, ?, ?, ?)''',
                                         (student_id, current_date, 'present', current_time, img_bytes))
                            conn.commit()
                            conn.close() # Always close the connection
                            marked_attendance_count += 1
                            status = "Present"
                        else:
                            status = "Already Marked"
                
            recognized_students_info.append({
                'id': student_id,
                'name': student_name,
                'course': course,
                'status': status,
                'time': current_time # Use the single current time for all detections in this image
            })

        return jsonify({
            'recognized': recognized_students_info,
            'marked_count': marked_attendance_count,
            'message': 'Attendance processing complete'
        })

    except Exception as e:
        print(f"Error during attendance marking: {e}")
        return jsonify({'error': f'An internal error occurred during attendance marking: {str(e)}'}), 500

@app.route('/get_students', methods=['GET'])
def get_students():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT student_id, first_name, last_name, email, course, year, registration_date FROM students")
    students = c.fetchall()
    conn.close()
    student_list = []
    for s_id, fname, lname, email, course, year, reg_date in students:
        student_list.append({
            'student_id': s_id,
            'first_name': fname,
            'last_name': lname,
            'email': email,
            'course': course,
            'year': year,
            'registration_date': reg_date
        })
    return jsonify(student_list)

@app.route('/get_student_image/<int:student_id>', methods=['GET'])
def get_student_image(student_id):
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT image FROM students WHERE student_id = ?", (student_id,))
    image_blob = c.fetchone()
    conn.close()
    if image_blob and image_blob[0]:
        return send_file(io.BytesIO(image_blob[0]), mimetype='image/jpeg')
    return jsonify({'error': 'Image not found for this student ID'}), 404

@app.route('/get_attendance_records', methods=['GET'])
def get_attendance_records():
    """Fetches all attendance records, ordered by date and time."""
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("""
        SELECT a.attendance_id, s.first_name, s.last_name, s.course, a.date, a.status, a.time
        FROM attendance a
        JOIN students s ON a.student_id = s.student_id
        ORDER BY a.date DESC, a.time DESC
    """)
    records = c.fetchall()
    conn.close()
    attendance_list = []
    for rec_id, fname, lname, course, date_str, status, time_str in records:
        attendance_list.append({
            'attendance_id': rec_id,
            'first_name': fname,
            'last_name': lname,
            'course': course,
            'date': date_str,
            'status': status,
            'time': time_str
        })
    return jsonify(attendance_list)

@app.route('/export_attendance', methods=['GET'])
def export_attendance():
    """Exports all attendance records as a CSV file."""
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("""
        SELECT s.first_name, s.last_name, s.course, a.date, a.status, a.time
        FROM attendance a
        JOIN students s ON a.student_id = s.student_id
        ORDER BY a.date DESC, a.time DESC
    """)
    records = c.fetchall()
    conn.close()

    # Create CSV content
    csv_buffer = io.StringIO()
    csv_buffer.write("First Name,Last Name,Course,Date,Status,Time\n")
    for record in records:
        csv_buffer.write(f'"{record[0]}","{record[1]}","{record[2]}","{record[3]}","{record[4]}","{record[5]}"\n')

    csv_output = csv_buffer.getvalue()
    
    # Send as a file
    return Response(
        csv_output,
        mimetype="text/csv",
        headers={"Content-disposition": "attachment; filename=attendance_records.csv"}
    )

@app.route('/get_analytics', methods=['GET'])
def get_analytics():
    """Provides attendance analytics based on course and date range."""
    course_filter = request.args.get('course', 'all')
    date_from_str = request.args.get('date_from')
    date_to_str = request.args.get('date_to')

    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()

    query = """
        SELECT s.course, a.date, a.status, COUNT(*)
        FROM attendance a
        JOIN students s ON a.student_id = s.student_id
        WHERE 1=1
    """
    params = []

    if course_filter != 'all':
        query += " AND s.course = ?"
        params.append(course_filter)
    
    if date_from_str:
        query += " AND a.date >= ?"
        params.append(date_from_str)
    
    if date_to_str:
        query += " AND a.date <= ?"
        params.append(date_to_str)
    
    query += " GROUP BY s.course, a.date, a.status ORDER BY a.date DESC, s.course"

    c.execute(query, params)
    raw_data = c.fetchall()
    conn.close()

    # Process raw data into a structured format for analytics
    analytics_data = {}
    for course, record_date, status, count in raw_data:
        if record_date not in analytics_data:
            analytics_data[record_date] = {}
        if course not in analytics_data[record_date]:
            analytics_data[record_date][course] = {'present': 0, 'absent': 0, 'total_students': 0}
        
        if status == 'present':
            analytics_data[record_date][course]['present'] += count
        # For simplicity, we assume if not present, they are absent for this count
        # A more robust system would track registered students who didn't mark present
        analytics_data[record_date][course]['total_students'] += count # This counts attendees, not total enrolled

    # To get actual absent counts, you would need to compare against total registered students
    # and those who were present. For basic analytics, we can derive it from present/total_attendees.
    
    # Further refine output for chart.js
    labels = sorted(analytics_data.keys()) # Dates
    courses = sorted(list(set([c for date_data in analytics_data.values() for c in date_data.keys()]))) # All courses

    datasets = {
        'present': {course: [] for course in courses},
        'absent': {course: [] for course in courses},
        'total': {course: [] for course in courses} # Total students detected/attended
    }

    for date_label in labels:
        for course in courses:
            data_for_date_course = analytics_data.get(date_label, {}).get(course, {'present': 0, 'absent': 0, 'total_students': 0})
            datasets['present'][course].append(data_for_date_course['present'])
            # For 'absent' here, we're assuming 'absent' means not marked present among those who could be.
            # This is an approximation. A true absent count would need to know total enrolled students for a given date.
            # For now, we'll just show 0 absent if only present data is available.
            datasets['absent'][course].append(data_for_date_course['absent']) # If you want to log explicit 'absent' status
            datasets['total'][course].append(data_for_date_course['total_students'])

    # Format for frontend
    formatted_response = {
        'labels': labels, # Dates
        'courses': courses,
        'data': {
            'present': [{ 'course': c, 'data': datasets['present'][c]} for c in courses],
            'absent': [{ 'course': c, 'data': datasets['absent'][c]} for c in courses],
            'total': [{ 'course': c, 'data': datasets['total'][c]} for c in courses]
        }
    }

    return jsonify(formatted_response)


if __name__ == '__main__':
    init_db()
    app.run(debug=False, port=5000)

In [ ]:
from flask import Flask, request, jsonify, send_file, Response
import sqlite3
import numpy as np
import os
import io
from datetime import datetime, date, timedelta
from flask_cors import CORS
from PIL import Image
from deepface import DeepFace
from scipy.spatial.distance import cosine

app = Flask(__name__)
CORS(app)

# --- Configuration for DeepFace ---
DEEPFACE_MODEL = 'VGG-Face'
DEEPFACE_THRESHOLD = 0.45

# Database initialization with improved schema
def init_db():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()

    # Create tables if they don't exist
    c.execute('''CREATE TABLE IF NOT EXISTS students
                 (student_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  first_name TEXT NOT NULL,
                  last_name TEXT NOT NULL,
                  email TEXT,
                  course TEXT,
                  year INTEGER,
                  face_encoding BLOB,
                  registration_date TEXT,
                  image BLOB)''')

    c.execute('''CREATE TABLE IF NOT EXISTS attendance
                 (attendance_id INTEGER PRIMARY KEY AUTOINCREMENT,
                  student_id INTEGER,
                  date TEXT,
                  status TEXT, -- 'present', 'absent', 'late'
                  time TEXT,
                  image BLOB, 
                  FOREIGN KEY(student_id) REFERENCES students(student_id))''')

    conn.commit()
    conn.close()

init_db()

# Helper functions for face encodings
def face_encoding_to_blob(encoding):
    return np.array(encoding, dtype=np.float32).tobytes()

def blob_to_face_encoding(blob):
    return np.frombuffer(blob, dtype=np.float32)

def load_known_faces():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT student_id, face_encoding FROM students WHERE face_encoding IS NOT NULL")
    rows = c.fetchall()
    conn.close()

    known_encodings = []
    known_ids = []
    for student_id, encoding_blob in rows:
        if encoding_blob:
            encoding_array = np.frombuffer(encoding_blob, dtype=np.float32)
            known_encodings.append(encoding_array)
            known_ids.append(student_id)
    return known_encodings, known_ids

def compare_deepface_encodings(known_encodings, unknown_encoding, threshold=DEEPFACE_THRESHOLD):
    matches = []
    for known_encoding in known_encodings:
        distance = cosine(known_encoding, unknown_encoding)
        matches.append(distance < threshold)
    return matches

# API Endpoints
@app.route('/register_student', methods=['POST'])
def register_student():
    try:
        data = request.form
        image_file = request.files.get('image')

        if not image_file:
            return jsonify({'error': 'No image file provided'}), 400

        img_bytes = image_file.read()
        pil_image = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        image_np = np.array(pil_image)

        detected_faces = DeepFace.extract_faces(
            img_path=image_np,
            detector_backend='opencv',
            enforce_detection=False
        )

        if not detected_faces:
            return jsonify({'error': 'No face detected in the image'}), 400
        elif len(detected_faces) > 1:
            return jsonify({'error': 'Multiple faces detected. Please upload an image with a single face.'}), 400

        face_area = detected_faces[0]['facial_area']
        x, y, w, h = face_area['x'], face_area['y'], face_area['w'], face_area['h']
        cropped_face_np = image_np[y:y+h, x:x+w]

        embeddings = DeepFace.represent(
            img_path=cropped_face_np,
            model_name=DEEPFACE_MODEL,
            enforce_detection=False
        )
        if not embeddings:
            return jsonify({'error': 'Could not generate face embedding.'}), 500

        face_encoding = embeddings[0]['embedding']
        face_blob = face_encoding_to_blob(face_encoding)

        known_face_encodings, _ = load_known_faces()
        if known_face_encodings:
            matches = compare_deepface_encodings(known_face_encodings, face_encoding, threshold=DEEPFACE_THRESHOLD)
            if any(matches):
                return jsonify({'error': 'This face is already registered in the system'}), 400

        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()

        c.execute('''INSERT INTO students
                    (first_name, last_name, email, course, year, face_encoding, registration_date, image)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                    (data.get('first_name'), data.get('last_name'), data.get('email'),
                     data.get('course'), data.get('year'), face_blob, datetime.now().isoformat(), img_bytes))

        student_id = c.lastrowid
        conn.commit()
        conn.close()

        return jsonify({
            'success': True,
            'student_id': student_id,
            'message': 'Student registered successfully with face data'
        })

    except sqlite3.IntegrityError as e:
        print(f"Database error during registration: {e}")
        return jsonify({'error': 'Database error. Student may already exist.'}), 400
    except Exception as e:
        print(f"Error during student registration: {e}")
        return jsonify({'error': f'An internal error occurred during registration: {str(e)}'}), 500

@app.route('/mark_attendance', methods=['POST'])
def mark_attendance():
    try:
        image_file = request.files.get('image')
        if not image_file:
            return jsonify({'error': 'No image file provided'}), 400

        img_bytes = image_file.read()
        pil_image = Image.open(io.BytesIO(img_bytes)).convert('RGB')
        unknown_image_np = np.array(pil_image)

        detected_faces_info = DeepFace.extract_faces(
            img_path=unknown_image_np,
            detector_backend='opencv',
            enforce_detection=False
        )

        if not detected_faces_info:
            return jsonify({'recognized': [], 'message': 'No faces detected in the image'}), 200

        unknown_face_encodings = []
        for face_info in detected_faces_info:
            x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']
            cropped_face_np = unknown_image_np[y:y+h, x:x+w]
            
            embeddings = DeepFace.represent(
                img_path=cropped_face_np,
                model_name=DEEPFACE_MODEL,
                enforce_detection=False
            )
            if embeddings:
                unknown_face_encodings.append(embeddings[0]['embedding'])

        if not unknown_face_encodings:
              return jsonify({'recognized': [], 'message': 'No recognizable faces found in the image'}), 200

        known_face_encodings, known_student_ids = load_known_faces()

        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()
        c.execute('SELECT student_id, first_name, last_name, course FROM students')
        students_data = {row[0]: (row[1], row[2], row[3]) for row in c.fetchall()}
        conn.close()

        recognized_students_info = []
        marked_attendance_count = 0
        current_date = date.today().isoformat()
        current_time = datetime.now().strftime("%H:%M:%S")

        for unknown_encoding in unknown_face_encodings:
            student_id = None
            student_name = "Unknown"
            course = "Unknown"
            status = "Not Recognized"

            if known_face_encodings:
                distances = [cosine(known_encoding, unknown_encoding) for known_encoding in known_face_encodings]
                min_distance_index = np.argmin(distances)
                min_distance = distances[min_distance_index]

                if min_distance < DEEPFACE_THRESHOLD:
                    student_id = known_student_ids[min_distance_index]
                    if student_id in students_data:
                        first_name, last_name, course = students_data[student_id]
                        student_name = f"{first_name} {last_name}"
                        
                        conn = sqlite3.connect('attendance.db')
                        c = conn.cursor()
                        c.execute('SELECT attendance_id FROM attendance WHERE student_id = ? AND date = ?',
                                  (student_id, current_date))
                        existing_attendance = c.fetchone()
                        conn.close()

                        if existing_attendance is None:
                            conn = sqlite3.connect('attendance.db')
                            c = conn.cursor()
                            c.execute('''INSERT INTO attendance
                                          (student_id, date, status, time, image)
                                          VALUES (?, ?, ?, ?, ?)''',
                                          (student_id, current_date, 'present', current_time, img_bytes))
                            conn.commit()
                            conn.close()
                            marked_attendance_count += 1
                            status = "Present"
                        else:
                            status = "Already Marked"
            
            recognized_students_info.append({
                'id': student_id,
                'name': student_name,
                'course': course,
                'status': status,
                'time': current_time
            })

        return jsonify({
            'recognized': recognized_students_info,
            'marked_count': marked_attendance_count,
            'message': 'Attendance processing complete'
        })

    except Exception as e:
        print(f"Error during attendance marking: {e}")
        return jsonify({'error': f'An internal error occurred during attendance marking: {str(e)}'}), 500

@app.route('/get_students', methods=['GET'])
def get_students():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT student_id, first_name, last_name, email, course, year, registration_date FROM students")
    students = c.fetchall()
    conn.close()
    student_list = []
    for s_id, fname, lname, email, course, year, reg_date in students:
        student_list.append({
            'student_id': s_id,
            'first_name': fname,
            'last_name': lname,
            'email': email,
            'course': course,
            'year': year,
            'registration_date': reg_date
        })
    return jsonify(student_list)

@app.route('/get_student_image/<int:student_id>', methods=['GET'])
def get_student_image(student_id):
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT image FROM students WHERE student_id = ?", (student_id,))
    image_blob = c.fetchone()
    conn.close()
    if image_blob and image_blob[0]:
        return send_file(io.BytesIO(image_blob[0]), mimetype='image/jpeg')
    return jsonify({'error': 'Image not found for this student ID'}), 404

@app.route('/get_recent_attendance', methods=['GET'])
def get_recent_attendance():
    """Returns recent attendance records with optional filters"""
    try:
        course_filter = request.args.get('course', 'all')
        date_from = request.args.get('date_from')
        date_to = request.args.get('date_to')
        limit = request.args.get('limit', '50')

        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()

        query = """SELECT a.attendance_id, s.student_id, s.first_name, s.last_name, 
                           s.course, a.date, a.status, a.time
                    FROM attendance a
                    JOIN students s ON a.student_id = s.student_id
                    WHERE 1=1"""
        params = []

        if course_filter != 'all':
            query += " AND s.course = ?"
            params.append(course_filter)
        
        if date_from:
            query += " AND a.date >= ?"
            params.append(date_from)
        
        if date_to:
            query += " AND a.date <= ?"
            params.append(date_to)
        
        query += " ORDER BY a.date DESC, a.time DESC LIMIT ?"
        params.append(int(limit))

        c.execute(query, params)
        records = c.fetchall()
        conn.close()

        attendance_list = []
        for rec_id, s_id, fname, lname, course, date_str, status, time_str in records:
            attendance_list.append({
                'attendance_id': rec_id,
                'student_id': s_id,
                'first_name': fname,
                'last_name': lname,
                'course': course,
                'date': date_str,
                'status': status,
                'time': time_str
            })

        return jsonify(attendance_list)

    except Exception as e:
        print(f"Error fetching recent attendance: {e}")
        return jsonify({'error': f'Error fetching attendance records: {str(e)}'}), 500

@app.route('/export_attendance', methods=['GET'])
def export_attendance():
    """Exports all attendance records as a CSV file."""
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("""
        SELECT s.first_name, s.last_name, s.course, a.date, a.status, a.time
        FROM attendance a
        JOIN students s ON a.student_id = s.student_id
        ORDER BY a.date DESC, a.time DESC
    """)
    records = c.fetchall()
    conn.close()

    csv_buffer = io.StringIO()
    csv_buffer.write("First Name,Last Name,Course,Date,Status,Time\n")
    for record in records:
        csv_buffer.write(f'"{record[0]}","{record[1]}","{record[2]}","{record[3]}","{record[4]}","{record[5]}"\n')

    csv_output = csv_buffer.getvalue()
    
    return Response(
        csv_output,
        mimetype="text/csv",
        headers={"Content-disposition": "attachment; filename=attendance_records.csv"}
    )

@app.route('/get_analytics', methods=['GET'])
def get_analytics():
    """Provides comprehensive attendance analytics"""
    try:
        course_filter = request.args.get('course', 'all')
        date_from_str = request.args.get('date_from')
        date_to_str = request.args.get('date_to')

        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()

        # Get total students count (for calculating absentees)
        total_students_query = "SELECT COUNT(*) FROM students"
        if course_filter != 'all':
            total_students_query += " WHERE course = ?"
            total_students_params = [course_filter]
        else:
            total_students_params = []
        
        c.execute(total_students_query, total_students_params)
        total_students = c.fetchone()[0]

        # Get attendance data
        query = """
            SELECT 
                a.date,
                s.course,
                COUNT(DISTINCT a.student_id) as present_count
            FROM 
                attendance a
            JOIN 
                students s ON a.student_id = s.student_id
            WHERE 
                a.status = 'present'
        """
        params = []

        if course_filter != 'all':
            query += " AND s.course = ?"
            params.append(course_filter)
        
        if date_from_str:
            query += " AND a.date >= ?"
            params.append(date_from_str)
        
        if date_to_str:
            query += " AND a.date <= ?"
            params.append(date_to_str)
        
        query += " GROUP BY a.date, s.course ORDER BY a.date"

        c.execute(query, params)
        attendance_data = c.fetchall()

        # Get all dates in the range for complete timeline
        date_query = "SELECT DISTINCT date FROM attendance WHERE 1=1"
        date_params = []
        
        if date_from_str:
            date_query += " AND date >= ?"
            date_params.append(date_from_str)
        
        if date_to_str:
            date_query += " AND date <= ?"
            date_params.append(date_to_str)
        
        date_query += " ORDER BY date"
        c.execute(date_query, date_params)
        all_dates = [row[0] for row in c.fetchall()]
        conn.close()

        # Process data for response
        analytics = {
            'total_students': total_students,
            'dates': all_dates,
            'courses': [],
            'present_data': [],
            'absent_data': [],
            'attendance_rate': []
        }

        # Group by course if no specific course filter
        if course_filter == 'all':
            course_data = {}
            for date, course, present_count in attendance_data:
                if course not in course_data:
                    course_data[course] = {d: 0 for d in all_dates}
                course_data[course][date] = present_count

            for course in course_data:
                analytics['courses'].append(course)
                analytics['present_data'].append({
                    'course': course,
                    'data': [course_data[course][d] for d in all_dates]
                })
                analytics['absent_data'].append({
                    'course': course,
                    'data': [total_students - course_data[course][d] for d in all_dates]
                })
                analytics['attendance_rate'].append({
                    'course': course,
                    'data': [round((course_data[course][d] / total_students) * 100, 1) if total_students > 0 else 0 
                             for d in all_dates]
                })
        else:
            # Single course view
            date_present_map = {d: 0 for d in all_dates}
            for date, course, present_count in attendance_data:
                date_present_map[date] = present_count

            analytics['courses'].append(course_filter)
            analytics['present_data'].append({
                'course': course_filter,
                'data': [date_present_map[d] for d in all_dates]
            })
            analytics['absent_data'].append({
                'course': course_filter,
                'data': [total_students - date_present_map[d] for d in all_dates]
            })
            analytics['attendance_rate'].append({
                'course': course_filter,
                'data': [round((date_present_map[d] / total_students) * 100, 1) if total_students > 0 else 0 
                         for d in all_dates]
            })

        # Calculate overall stats
        if all_dates:
            latest_date = max(all_dates)
            latest_present = sum([d['data'][-1] for d in analytics['present_data']])
            latest_absent = total_students - latest_present
            latest_rate = round(latest_present / total_students * 100, 1) if total_students > 0 else 0
            
            analytics['overall_stats'] = {
                'latest_date': latest_date,
                'present': latest_present,
                'absent': latest_absent,
                'rate': latest_rate,
                'most_absent': "N/A"  # This would require additional query to find students with most absences
            }

        return jsonify(analytics)

    except Exception as e:
        print(f"Error generating analytics: {e}")
        return jsonify({'error': f'Error generating analytics: {str(e)}'}), 500

if __name__ == '__main__':
    init_db()
    app.run(debug=False, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2025 02:30:14] "POST /register_student HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2025 02:30:57] "POST /register_student HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2025 02:31:04] "GET /get_recent_attendance HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2025 02:34:07] "POST /mark_attendance HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2025 02:34:07] "GET /get_recent_attendance HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2025 02:34:25] "GET /get_analytics?course=all&date_from=2025-05-23&date_to=2025-06-23 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2025 02:34:25] "GET /get_recent_attendance?course=all&date_from=2025-05-23&date_to=2025-06-23 HTTP/1.1" 200 -
